In [1]:
import json
import os
import engine
import numpy as np
import random
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
from num2words import num2words
from IPython.display import display
from PIL import Image
from pprint import pprint
from tqdm import tqdm
from functools import reduce

In [2]:
generator = engine.QGenerator()
data_prefix = 'val'
scenes = json.load(open(f'{data_prefix}_sceneGraphs.json', 'r'))
valid_questions = json.load(open('gqa_questions/valid.json'))
img_ids = set(json.load(open('valid_img_ids.json')))
scenes = {k: scenes[k] for k in list(scenes.keys()) for k in img_ids}

In [3]:
assert set(scenes.keys()) == img_ids
vocab = set(json.load(open('val_label_vocab.json')))

In [10]:
def get_questions(question_family, scene):
    family = json.load(open(f'question_families/{question_family}.json', 'r'))
    tokens = frozenset(family['tokens'])
    constraints = set(family['constraints'])
    program = family['program']
    templates = family['templates']
    all_assignments = list()
    for assignment in generator.generate_assignments(scene, tokens, constraints):
        if assignment not in all_assignments:
            all_assignments.append(assignment)
    sent_ans = set()
    for a_ix, assignment in enumerate(all_assignments):
        answer = generator.handler.get_answer(scene, program, assignment)
        for t_ix, template in enumerate(templates):
            sent = generator.expand_text_template(template, assignment)
            sent_ans.add((sent, answer))
    return list(zip(sent_ans))

In [16]:
img_id_menu = widgets.Dropdown(
    options=list(map(str, sorted(map(int, scenes.keys())))),
    description='Image:',
    disabled=False,
)
question_families = list(map(lambda x: x.rsplit('.')[0], 
                             filter(lambda x: x.endswith('.json'), 
                                    os.listdir('question_families/'))))
question_families.insert(0, 'gqa')
question_family_menu = widgets.Dropdown(
    options=question_families,
    description='Q Family:',
    disabled=False,
)
def show_image(value):
    img_filename = f'./images/{value}.jpg'
    im = Image.open(img_filename)
    ratio = 500/im.size[0]
    im = im.resize([round(im.size[0]*ratio), round(im.size[1] * ratio)])
    display(im)
def show_questions(family, scene_id):
    scene = scenes[scene_id]
    show_image(scene_id)
    if family != 'gqa':
        for sent_answer in get_questions(family, scene):
            sent, answer = sent_answer[0]
            print(sent)
            print(f'  {answer}')
    else:
        questions = list(filter(lambda x: x['img_id'] == scene_id, valid_questions))
        for question in questions:
            sent = question['sent']
            answer = list(question['label'].keys())[0]
            print(sent)
            print(f'  {answer}')

img_q = widgets.interactive(show_questions, family=question_family_menu, scene_id=img_id_menu)
display(img_q)

interactive(children=(Dropdown(description='Q Family:', options=('gqa', 'color_query_bool', 'color_query', 'si…